In [47]:
import sys
import pandas as pd
import boto3
from time import time
import subprocess
sys.path.append('../src')
import cb_utils
import datetime
import json

pd.options.display.max_columns = 500
con = cb_utils.get_engine()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
bucket_name = 'cb-member-doc-us-east-2-prd'
prefix = 'parsed_claim_file_archive/'

In [4]:
client = boto3.client('s3')

In [5]:
response = client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

In [7]:
contents = response['Contents']

In [52]:
s3 = boto3.resource('s3')
bucket = s3.Bucket(bucket_name)    
files = []
for file in bucket.objects.filter(Prefix=prefix):
    #compare dates 
    if (file.last_modified).replace(tzinfo = None).date() >= datetime.datetime(2021,1, 5,tzinfo = None).date():
        files.append(file)

In [53]:
len(files)

551

In [54]:
files = [f for f in files if '999.x12.json' not in f.key]

In [55]:
len(files)

365

In [56]:
files = [f for f in files if '.277Ack.x12.json' not in f.key]

In [57]:
len(files)

189

In [58]:
files = [f for f in files if '.277.json' in f.key]

In [59]:
len(files)

35

In [60]:
hs_files = [
'10472_UB920166_2021010406020901.277.json',
'10652_UB920166_2021010501521901.277.json',
'10653_UB920166_2021010502035101.277.json',
'10654_UB920166_2021010502071001.277.json',
'10655_UB920166_2021010502194201.277.json',
'10656_UB920166_2021010502411601.277.json',
'10657_UB920166_2021010504160001.277.json',
'10658_UB920166_2021010504160002.277.json',
'10659_UB920166_2021010504160003.277.json',
'10660_UB920166_2021010504160004.277.json',
'10661_UB920166_2021010504160005.277.json',
'10662_UB920166_2021010504160006.277.json',
'10663_UB920166_2021010504160101.277.json',
]
def hs_file(f):
    is_hs_file = False
    for hs_f in hs_files:
        if hs_f in f.key:
            is_hs_file = True
            break
    return is_hs_file

In [61]:
files = [f for f in files if not hs_file(f)]

In [62]:
len(files)

23

In [36]:
for f in files[:23]:
    print(f.key, f.last_modified)

parsed_claim_file_archive/10474_GATE0166_2021010413064501.277.json 2021-01-05 00:00:22+00:00
parsed_claim_file_archive/10475_GATE0166_2021010416160401.277.json 2021-01-05 00:00:23+00:00
parsed_claim_file_archive/10476_GATE0166_2021010420005201.277.json 2021-01-05 06:00:23+00:00
parsed_claim_file_archive/10735_UB920166_2021010506015101.277.json 2021-01-05 18:00:25+00:00
parsed_claim_file_archive/10736_UB920166_2021010506160101.277.json 2021-01-05 18:00:25+00:00
parsed_claim_file_archive/10737_UB920166_2021010507160301.277.json 2021-01-05 18:00:26+00:00
parsed_claim_file_archive/10739_GATE0166_2021010511524401.277.json 2021-01-06 00:00:24+00:00
parsed_claim_file_archive/10740_GATE0166_2021010512312601.277.json 2021-01-06 00:00:25+00:00
parsed_claim_file_archive/10741_GATE0166_2021010516160501.277.json 2021-01-06 00:00:25+00:00
parsed_claim_file_archive/10742_UB920166_2021010516020001.277.json 2021-01-06 00:00:26+00:00
parsed_claim_file_archive/10743_GATE0166_2021010520080901.277.json 202

In [37]:
potential = [
'parsed_claim_file_archive/10735_UB920166_2021010506015101.277.json',
'parsed_claim_file_archive/10736_UB920166_2021010506160101.277.json',
'parsed_claim_file_archive/10737_UB920166_2021010507160301.277.json',
]

In [38]:
for f in files:
    if f.key in potential:
        print(f)

s3.ObjectSummary(bucket_name='cb-member-doc-us-east-2-prd', key='parsed_claim_file_archive/10735_UB920166_2021010506015101.277.json')
s3.ObjectSummary(bucket_name='cb-member-doc-us-east-2-prd', key='parsed_claim_file_archive/10736_UB920166_2021010506160101.277.json')
s3.ObjectSummary(bucket_name='cb-member-doc-us-east-2-prd', key='parsed_claim_file_archive/10737_UB920166_2021010507160301.277.json')


In [70]:
content = [(f, f.get()['Body'].read().decode('utf-8')) for f in files]

In [71]:
# json.loads?
# c = content[0]['Body'].read().decode('utf-8')

In [74]:
for f, c in content:
    if '100000035' in c:
        if '3785' in c:
            print('one', f)
            file_content_3785 = json.loads(c)
        if '3778' in c:
            print('two', f)
            file_content_3778 = json.loads(c)

one s3.ObjectSummary(bucket_name='cb-member-doc-us-east-2-prd', key='parsed_claim_file_archive/10739_GATE0166_2021010511524401.277.json')
two s3.ObjectSummary(bucket_name='cb-member-doc-us-east-2-prd', key='parsed_claim_file_archive/10739_GATE0166_2021010511524401.277.json')


In [75]:
file_content_3778

{'document': [{'billing_provider_name': 'CAREBRIDGE MEDICAL GROUP, PC',
   'billing_provider_npi': '1124656616',
   'check_date': None,
   'check_number': '',
   'claim_amount': '0',
   'claim_external_status_code': 'A6:21',
   'claim_external_status_description': 'Missing or invalid information. Note: At least one other status code is required to identify the missing or invalid information.',
   'claim_status': 'U',
   'clearinghouse_id': None,
   'date_of_service': '2020-10-13',
   'date_of_services': '20201013-20201013',
   'is_claim_accepted': False,
   'is_claim_denied': False,
   'is_claim_paid': False,
   'is_claim_rejected': True,
   'is_submitter_accepted': True,
   'paid_amount': '',
   'patient_control_num': '3778',
   'patient_fname': 'SHIRLEY',
   'patient_id': '106538691',
   'patient_lname': 'LINCOLN',
   'payer_clrhs_name': 'INGENIX - EDIS',
   'payer_entity_identifier_code': 'AY',
   'payer_entity_type_qualifier': '2',
   'payer_identifier': '9080180109',
   'process_d